In [28]:
import os
import requests
from urllib.parse import urljoin
from llama_index.core import PromptTemplate, VectorStoreIndex
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.readers.web import SimpleWebPageReader

import uuid

from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_postgres import PostgresChatMessageHistory
import psycopg

from langchain_cohere import CohereEmbeddings
from langchain_postgres import PGVector
from langchain_postgres.vectorstores import PGVector
from langchain_core.documents import Document

In [ ]:
model = Ollama(model='llama3')

In [48]:
DB_NAME: str = os.environ.get('DB_NAME')
DB_USER: str = os.environ.get('DB_USER')
DB_HOST: str = os.environ.get('DB_HOST')
DB_PORT: str = os.environ.get('DB_PORT')
DB_PW: str = os.environ.get('DB_PW')
conn_info = f"postgresql://{DB_USER}:{DB_PW}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
sync_connection = psycopg.connect(conninfo=conn_info)

table_name = "chat_history"
PostgresChatMessageHistory.create_tables(sync_connection, table_name)

In [10]:
session_id = str(uuid.uuid4())

chat_history = PostgresChatMessageHistory(
    table_name,
    session_id,
    sync_connection=sync_connection
)

# Add messages to the chat history
chat_history.add_messages([
    SystemMessage(content="Meow"),
    AIMessage(content="woof"),
    HumanMessage(content="bark"),
])

print(chat_history.messages)

[SystemMessage(content='Meow'), AIMessage(content='woof'), HumanMessage(content='bark')]


In [11]:
human_message = "Tell me a joke"
ai_message = model.invoke(human_message)
system_message = "How can i help you?"

chat_history.add_messages([
    SystemMessage(content=system_message),
    AIMessage(content=ai_message),
    HumanMessage(content=human_message),
])

print(chat_history.messages)

[SystemMessage(content='Meow'), AIMessage(content='woof'), HumanMessage(content='bark'), SystemMessage(content='How can i help you?'), AIMessage(content="Here's one:\n\nWhy don't eggs tell jokes?\n\n(Wait for it...)\n\nBecause they'd crack each other up!\n\nHope that made you smile!"), HumanMessage(content='Tell me a joke')]


In [44]:

class ReardenRAG:
    def __init__(self, urls_files_path: list):
        all_urls = self.get_all_urls(base_urls_files=urls_files_path)        
        self.documents = SimpleDirectoryReader(os.path.join("..", "rearden_llm", "knowledge")).load_data()
        self.urls = SimpleWebPageReader(html_to_text=True).load_data(urls=all_urls)
        self.documents+= self.urls
        self.rerank = SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3)
        self.embed_model = OllamaEmbedding(model_name="nomic-embed-text")    
        Settings.embed_model = self.embed_model
        Settings.chunk_size = 1048    
        Settings.llm = Ollama(model="llama3", request_timeout=360.0)
        index = VectorStoreIndex.from_documents(
            self.documents,
        )
        self.query_engine = index.as_query_engine()

        # Settings.chunk_size = 512

    def get_all_urls(self, base_urls_files: list):
        urls = []
        for _file in base_urls_files:
            with open(_file, "r") as file:
                lines = file.readlines()
                lines = [line.strip() for line in lines]
            urls += lines
        return urls

    def test(self, message: str):
        res = self.query_engine.query(message)
        print(res)
        return res.response
    
url_files = [os.path.join("..","rearden_llm","urls", "starknet_urls.txt"), 
             os.path.join("..","rearden_llm","urls", "wormhole_urls.txt"), 
             os.path.join("..","rearden_llm","urls", "sui_urls.txt")]
    
rag_llm = ReardenRAG(urls_files_path=url_files)

/srv/rearden/dev/back/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [49]:
session_id = str(uuid.uuid4())
chat_history = PostgresChatMessageHistory(
        table_name,
        session_id,
        sync_connection=sync_connection
    )
question_list = ['What is sui?', 
                 'What is starknet?', 
                 'What is wormhole?', 
                 'How can i interact with sui?',
                 'What main types of participants characterize the Sui economy',
                 'How to setup my environment with starknet?',
                 'Which architect starknet use?',
                 'What is StarkGate?',
                 'How to develop Cross Chain Dapps?',
                 'What environment wormhole supports?']
for question in question_list: 
    
    # Add messages to the chat history
    chat_history.add_messages([
        AIMessage(content=rag_llm.test(message=question)),
        HumanMessage(content=question),
    ])

Based on the provided context information, Sui refers to a platform.
Starknet is a ZK-proving system that uses Poseidon hash functions to be very efficient as algebraic circuits.
Wormhole is a system that enables interoperability between different blockchain networks.
You can interact with Sui by visiting their platform at [Sui overview](/intro-to-sui) or by exploring other options such as:

* Move: [Move](/move)
* zkLogin: [zklogin](/zklogin)
* Use cases: [Use cases](/use-cases)
* Research: [Research](/research)
* Network status: (https://status.sui.io/)
* Network information: (/networkinfo)
* Token schedule: (/token-schedule)
* Bug Bounty Program: (/bug-bounty-program)

Additionally, you can engage with the Sui ecosystem through:

* Global events: (/community-events-hub#events)
* Community programs: (/community-events-hub#programs)
* Job board: (https://jobs.sui.io/jobs)
* Directory: (https://sui.directory/)
* Sui University Initiative: (https://sui.io/community-access-program)

You 